# Set environment

In [ ]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [ ]:
!git clone https://github.com/gilbertolem/ProgGen

In [ ]:
from os import chdir, getcwd
chdir("ProgGen")
from sys import path
path.append(getcwd())

# Train

In [ ]:
import torch
import utils.data_tools as data_tools
from utils.nets import ProgGen
from pickle import load, dump
import matplotlib.pyplot as plt

xml_directory = "XML_Tunes/"
use_gpu = torch.cuda.is_available()

In [ ]:
# Load vocabulary
words_text2num = load(open("maps/words_text2num.txt",'rb'))
vocab_size = len(words_text2num)

# Create training data
mode = "all_keys"
filters = {'author':'Charlie Parker', 'style':None}
X = data_tools.musicxml2tensor(xml_directory, words_text2num, mode = mode, filters = filters) # (Seq x Batch x vocab_size)
data = data_tools.TuneData(X)
loader = torch.utils.data.DataLoader(data, batch_size = 372, num_workers = 1 if use_gpu else 4)

In [ ]:
print(X.size())

In [ ]:
# Construct Neural Net
input_size = vocab_size
hidden_size = 256
num_layers = 1
dropout = 0
model = ProgGen(input_size, hidden_size, num_layers, dropout)
loss_fn = torch.nn.CrossEntropyLoss()

if use_gpu:
    model = model.cuda()
    loss_fn = loss_fn.cuda()

In [ ]:
# Define loss function and optimizer
lr = 1e-2
optim = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
from utils.training import train
epochs = 200
losses = train(epochs, model, optim, loader, loss_fn, use_gpu)

In [ ]:
model_name = "model"
dump(model, open('models/'+model_name+'.dat','wb'))
plt.semilogy(losses)
plt.show()

# Generate something

In [ ]:
from utils.generating import generate_progression

model_name = "model"
initial_chord = "4C_maj"
tune_len = 32
top = 1

generate_progression(initial_chord, tune_len, top, model_name)